# Lesson 4: Call transcript summarizer

このレッスンでは、実務でよくある「要約」ユースケース向けに、複雑なプロンプトを書きます。具体的には、カスタマーサービスの通話記録（call transcript）を要約します。

目的は、サポートチームの有効性を評価するための **有用で構造化されたデータ** として、通話内容をまとめることです。そのため、接続不良・言語障壁など、要約しても意味が薄い通話は除外したいです。

ここでは、スマートホーム機器を販売する架空企業「Acme Corporation」で働いていると想像してください。同社は毎日何百件ものサポート通話を扱っており、会話を素早く構造化データへ変換したい、という状況です。

重要な考慮点：
* 通話は短いものも長いものもある
* Wi‑Fi接続の簡単な問題から、複雑な故障まで多様な問い合わせがある
* 後で分析しやすいよう、要約は特定フォーマットで欲しい
* 個人情報は要約に含めないよう注意する

これまでに扱ったベストプラクティスに沿って進めます：
* system prompt で前提と役割を定める
* 最高性能を出しやすい構造でプロンプトを組む
* 明確な指示と望ましい出力を定義する
* XMLタグで情報を整理する
* 特殊ケース・エッジケースを扱う
* 例（examples）でモデルを誘導する

---


## Understanding the data

タスクが分かったところで、扱うデータを見てみます。このレッスンでは、Acme のスマートホーム機器サポートチームの「シミュレーション通話記録」を複数用意し、さまざまな状況に耐えるプロンプトを作ります。

想定しうる通話記録の例を確認しましょう。


短くシンプルな通話記録：


In [ ]:
call1 = """
エージェント: アクリーム・スマートホームサポートへようこそ。アレックスです。どのようにお手伝いできますか？
顧客: こんにちは、スマート電球が点灯しません。
エージェント: 承知いたしました。電球をリセットしてみましたか？
顧客: いえ、まだです。どうすればいいですか？
エージェント: 電源を5秒間オフにしてから、再度オンにしてください。リセットされるはずです。
顧客: わかりました。試してみます。ありがとうございます！
エージェント: どういたしまして。他にご不明な点がございましたら、お気軽にお電話ください。
"""

中くらいの長さで、最終的に解決に至る通話記録：


In [ ]:
call2 = """
エージェント: アクリーム・スマートホームサポートです。ジェイミーと申します。本日はどのようにお手伝いできますか？
顧客: こんにちは、ジェイミーさん。アクリーム・スマートサーモが設定した温度を維持できていません。72度に設定しているのに、家の中がもっと暑いんです。
エージェント: それは申し訳ございません。トラブルシューティングを始めましょう。スマートサーモはWi-Fiに接続されていますか？
顧客: はい、ディスプレイにWi-Fiのマークが表示されています。
エージェント: 承知いたしました。スマートサーモを再キャリブレーションしましょう。メニューボタンを5秒間長押ししてください。
顧客: わかりました。新しいメニューが表示されました。
エージェント: 完璧です。「キャリブレーション」に移動して選択を押してください。部屋の温度計に合わせて温度を調整してください。
顧客: 了解しました。79度に合わせて設定しました。
エージェント: よろしいです。選択を押して確定してください。再キャリブレーションには数分かかる場合があります。1時間後に確認して、修正されているかご確認ください。
顧客: わかりました。そうします。ご協力ありがとうございました、ジェイミーさん。
エージェント: どういたしまして！他に何かお手伝いできることはありますか？
顧客: いえ、それで大丈夫です。ありがとうございました。
エージェント: アクリーム・スマートホームをご利用いただき、ありがとうございます。良い一日をお過ごしください！
"""

長めで、解決に至らない通話記録：


In [ ]:
call3 = """
エージェント: アクリーム・スマートホームサポートへお電話ありがとうございます。サラと申します。本日はどのようにお手伝いできますか？
顧客: こんにちは、サラさん。アクリーム・セキュアホームシステムに問題があります。アラームがランダムに鳴り続けています。
エージェント: それは申し訳ございません。いつ頃から発生していますか？
顧客: 2日前くらいからです。今までに3回鳴りました。いつも真夜中です。
エージェント: 承知いたしました。アラームが鳴った時に、コントロールパネルにエラーメッセージは表示されていましたか？
顧客: いえ、気づきませんでした。その時はかなり眠かったので。
エージェント: 了解しました。いくつか確認させてください。まず、すべてのドアと窓が正しく閉まっているか確認していただけますか？
顧客: はい、すべて確認しました。問題ありません。
エージェント: わかりました。次に、コントロールパネルのバッテリーを確認しましょう。バッテリー残量が少ないインジケーターが点灯していますか？
顧客: 少々お待ちください...いえ、バッテリーインジケーターは正常に見えます。
エージェント: 承知いたしました。センサーのいずれかが故障している可能性があります。診断を実行したいのですが、そのためには技術チームにおつなぎする必要があります。よろしいですか？
顧客: はい、お願いします。早く直してほしいです。本当に迷惑です。
エージェント: お気持ちはよくわかります。今すぐおつなぎします。技術チームがシステム全体の診断を実行し、問題を解決できるはずです。
顧客: わかりました。ありがとうございます。
エージェント: どういたしまして。お待たせして申し訳ございませんでした。良い一日をお過ごしください。
"""

これらの例から、考慮すべき点が分かります：
* 通話の長さが大きく異なる
* 問い合わせ内容が多様（簡単な修正、故障、複雑な問題）
* 解決で終わる通話もあれば未解決で終わる通話もある
* フォローアップが必要な通話がある

この多様性に対応しつつ、重要情報を抽出して、一貫した構造で出力できるプロンプトが必要です。

次のセクションでは、ステップ・バイ・ステップでプロンプトを組み立てていきます。

---


## A simple version of the prompt

タスクとデータのイメージが掴めたので、プロンプトを作り始めます。まずは基本版から始め、徐々に改善して複雑さに対応します。

まずは、タスクの概要だけを書いた非常にシンプルなプロンプトから：


In [ ]:
prompt = """
以下のカスタマーサービス通話記録を要約してください。主な問題、解決方法、必要なフォローアップに焦点を当ててください。

{transcript}
"""

この基本プロンプトでも大枠は伝わりますが、次の欠点があります：

* 出力フォーマットを指定していないため、要約が不安定になりやすい
* 未解決や情報不足など、状況別の扱いが指示されていない
* 長さや含める内容の制約がなく、冗長になりやすい
* 個人情報を省く指示がなく、プライバシー問題を起こしうる

まずは性能の感触を掴むため、実行してみましょう。


In [46]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
client = Anthropic()

def summarize_call(transcript):
    final_prompt = prompt.format(transcript=transcript)
    # Make the API call
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    print(response.content[0].text)

In [47]:
summarize_call(call1)

Here is a summary of the customer service call transcript:

Main Issue:
The customer was unable to turn on their Acme smart light bulb.

Resolution:
The service agent instructed the customer to reset the bulb by turning the power off for 5 seconds and then back on. This should reset the bulb and allow it to turn on.

Follow-Up:
The agent told the customer to call back if they continued to have issues after trying the reset procedure. No other follow-up was mentioned.


In [48]:
summarize_call(call2)

Summary:

Main Issue: The customer's Acme SmartTherm thermostat was not maintaining the set temperature of 72°F, and the house was much warmer.

Resolution: The agent guided the customer through the process of recalibrating the SmartTherm thermostat. This involved accessing the "Calibration" menu, adjusting the temperature to match the customer's room thermometer (79°F in this case), and confirming the new setting. The recalibration process may take a few minutes to complete.

Follow-up Required: The customer was advised to check the thermostat in an hour to see if the issue was resolved after the recalibration process completed.


In [49]:
summarize_call(call3)

Here is a summary of the customer service call transcript:

Main Issue:
The customer was having an issue with their Acme SecureHome alarm system going off randomly in the middle of the night, even though all doors and windows were closed properly.

How It Was Resolved:
The customer service agent first had the customer check for any error messages on the control panel and confirm that the battery was not low. When those basic troubleshooting steps did not reveal the issue, the agent determined that one of the sensors may be malfunctioning and needed to transfer the customer to the technical support team for a full system diagnostic.

Required Follow-Up:
The technical support team needs to run a diagnostic on the customer's SecureHome system to identify which sensor(s) may be causing the false alarms and then repair or replace those components. The customer should be contacted again once the diagnostic is complete and the repair/replacement has been performed to ensure the random alarms 

Claude は要約を返してくれますが、体系的に分析しやすい形式ではありません。長すぎたり短すぎたり、注目したい点が毎回揃わなかったりします。

次からは、構造とガイダンスを足してこれらの問題を解消していきます。追加のたびに、品質と一貫性が改善するのを確認します。

プロンプトエンジニアリングは反復プロセスです。まず単純に始めて、徐々に洗練させます。

---


## Adding a system prompt

最初の改善として、system prompt で全体の文脈と役割を定めます。これにより、Claude の振る舞いの方向性が揃いやすくなります。

まずは次の system prompt から始めます：


In [ ]:
system = """
あなたは、通話記録から重要な情報を抽出し、構造化された形式で要約することに長けた専門のカスタマーサービス分析者です。
あなたのタスクは、カスタマーサービス通話記録を分析し、専門的なトーンを保ちながら、簡潔で正確な要約を生成することです。
"""

---

## Structuring our main prompt

次にメインプロンプトを書きます。ここでは次のヒントを使います：

- 長文ドキュメント（通話記録）は上の方に置く
- 詳細な指示と出力フォーマット要件を与える
- XMLタグで入力と出力を構造化する
- Claude に「声に出して考える」余地を与える

プロンプトが長くなるので、部品ごとに書いてから結合します。


### The input data

Claude のようなLLMでは、通話記録のような長文はプロンプトの冒頭に置くのが基本です。そうすることで、指示を読む前に必要な文脈が揃います。

また、プロンプト内で通話記録を識別できるように XML タグも使います：


In [ ]:
prompt_pt1 = """
以下のカスタマーサービス通話記録を分析し、対話のJSON要約を生成してください：

<transcript>
[通話記録をここに挿入]
</transcript>
"""

### Instructions and output format

ここで、望ましい構造化出力を明確にしましょう。解析しやすくするには JSON にするのが楽です。では、このケースで良い JSON とは何でしょう？

最低限、次の項目は欲しいです：
- 要約できるだけの情報があるかどうかのステータス（後で扱います。ひとまず全件 "COMPLETE" とします）
- 顧客の主な問題の要約
- 追加フォローアップが必要か
- 必要ならフォローアップ内容（折り返し連絡等）
- 解決方法（あれば）
- 会話の曖昧点（ambiguities）の一覧

JSON 構造案：

```json
{
  "summary": {
    "customerIssue": "Brief description of the main problem or reason for the call",
    "resolution": "How the issue was addressed or resolved, if applicable",
    "followUpRequired": true/false,
    "followUpDetails": "Description of any necessary follow-up actions, or null if none required"
  },
  "status": "COMPLETE",
  "ambiguities": ["List of any unclear or vague points in the conversation, or an empty array if none"]
}
```

これを踏まえ、プロンプトに入れる指示を作ります：
- 主な問題、解決、フォローアップを中心に要約する
- 指定の JSON 形式で出力する
- 個人情報は要約に含めない
- 各項目は短くまとめる

以下が出力指示（JSON形式を含む）の例です：


In [ ]:
prompt_pt2 = """
指示:
1. 通話記録を注意深く読んでください。
2. 通話記録を分析し、主な問題、解決方法、必要なフォローアップに焦点を当ててください。
3. 指定された構造に従って、対話の主要な側面を要約するJSONオブジェクトを生成してください。

重要なガイドライン:
- 機密性: 名前、電話番号、メールアドレスなどの特定の顧客データはすべて省略してください。
- 文字数制限: 各テキストフィールドは最大100文字に制限してください。
- 要約では専門的なトーンを保ってください。

出力形式:
以下の構造を持つJSONオブジェクトを生成してください：
<json>
{
  "summary": {
    "customerIssue": "主な問題または通話の理由の簡潔な説明",
    "resolution": "問題がどのように対処または解決されたか（該当する場合）",
    "followUpRequired": true/false,
    "followUpDetails": "必要なフォローアップアクションの説明、または不要な場合はnull"
  },
  "status": "COMPLETE",
  "ambiguities": ["会話の不明確または曖昧な点のリスト、または該当しない場合は空の配列"]
}
</json>
"""

---

## Using XML tags and giving Claude room to think

次に、2つの戦略を追加します：
- `<thinking>` タグで分析（考察）を先に出させる
- JSON 出力は `<json>` タグ内に入れさせる

これが初稿プロンプトの最後のピースです：


In [ ]:
prompt_pt3 = """
JSONを生成する前に、<thinking>タグ内で通話記録を分析してください。
主な問題、解決方法、フォローアップ要件、および曖昧な点の特定を含めてください。
その後、<json>タグ内にJSON出力を提供してください。
"""

`<thinking>` の中で分析させることで、最終的な JSON を作る前に思考を分解させられます。結果として、より丁寧で構造化された分析になりやすいです。

`<thinking>` を残すことで、レビューやデバッグ、品質保証の観点でも、モデルがどう判断したかを追いやすくなります。

分析（`<thinking>`）と構造化出力（`<json>`）を分離することで、「解釈」と「最終要約」を明確に分けられます。また `<json>` 内に JSON を閉じ込めることで、最終レスポンスから JSON だけを簡単に抽出できます。


---

## Testing our updated prompt

ここまでに書いた各部品を結合して、プロンプト全体を作ったものが次です：


In [ ]:
system = """
あなたは、通話記録から重要な情報を抽出し、構造化された形式で要約することに長けた専門のカスタマーサービス分析者です。
あなたのタスクは、カスタマーサービス通話記録を分析し、専門的なトーンを保ちながら、簡潔で正確な要約を生成することです。
"""

prompt = """
以下のカスタマーサービス通話記録を分析し、対話のJSON要約を生成してください：

<transcript>
[通話記録をここに挿入]
</transcript>

指示:
1. 通話記録を注意深く読んでください。
2. 通話記録を分析し、主な問題、解決方法、必要なフォローアップに焦点を当ててください。
3. 指定された構造に従って、対話の主要な側面を要約するJSONオブジェクトを生成してください。

重要なガイドライン:
- 機密性: 名前、電話番号、メールアドレスなどの特定の顧客データはすべて省略してください。
- 文字数制限: 各テキストフィールドは最大100文字に制限してください。
- 要約では専門的なトーンを保ってください。

出力形式:
以下の構造を持つJSONオブジェクトを生成してください：
<json>
{
  "summary": {
    "customerIssue": "主な問題または通話の理由の簡潔な説明",
    "resolution": "問題がどのように対処または解決されたか（該当する場合）",
    "followUpRequired": true/false,
    "followUpDetails": "必要なフォローアップアクションの説明、または不要な場合はnull"
  },
  "status": "COMPLETE",
  "ambiguities": ["会話の不明確または曖昧な点のリスト、または該当しない場合は空の配列"]
}
</json>

JSONを生成する前に、<thinking>タグ内で通話記録を分析してください。
主な問題、解決方法、フォローアップ要件、および曖昧な点の特定を含めてください。
その後、<json>タグ内にJSON出力を提供してください。
"""

プロンプトをテストするための関数例：


In [ ]:
def summarize_call_with_improved_prompt(transcript):
    final_prompt = prompt.replace("[通話記録をここに挿入]", transcript)
    # API呼び出しを実行
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        system=system,
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    print(response.content[0].text)

先ほど定義した通話記録のいくつかで試してみましょう：


In [58]:
summarize_call_with_improved_prompt(call1)

<thinking>
From the transcript, the main issue appears to be that the customer could not turn on their smart light bulb. The resolution provided by the agent was to reset the bulb by turning the power off for 5 seconds and then back on.

The agent did offer for the customer to call back if they needed further assistance, indicating potential follow-up may be required if the reset did not resolve the issue. However, no specific follow-up details were provided.

There do not seem to be any significant ambiguities in the conversation.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Unable to turn on smart light bulb",
    "resolution": "Agent instructed customer to reset the bulb by turning power off for 5 seconds, then back on",
    "followUpRequired": true,
    "followUpDetails": "Customer was advised to call back if the reset did not resolve the issue"
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>


In [59]:
summarize_call_with_improved_prompt(call2)

<thinking>
Main issue: The customer's Acme SmartTherm thermostat is not maintaining the set temperature of 72°F, and the house is much warmer.

Resolution: The agent guided the customer through recalibrating the SmartTherm thermostat by:
1. Having the customer press and hold the menu button for 5 seconds.
2. Navigating to the "Calibration" menu and selecting it.
3. Adjusting the temperature to match the customer's room thermometer reading of 79°F.
4. Confirming the new calibration setting.

Follow-up required: Yes, the agent instructed the customer to check back in an hour to see if the recalibration resolved the temperature issue.

Ambiguities: None
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Thermostat not maintaining set temperature, causing house to be much warmer.",
    "resolution": "Agent guided customer through recalibrating the thermostat to match room temperature.",
    "followUpRequired": true,
    "followUpDetails": "Customer to check back in an hour to see i

In [60]:
summarize_call_with_improved_prompt(call3)

<thinking>
Main issue: The customer's Acme SecureHome system alarm is going off randomly in the middle of the night, even though doors and windows are closed properly.

Resolution: The agent suggests running a diagnostic on the system to identify potential sensor malfunctions. The customer is transferred to the technical team to perform the diagnostic and resolve the issue.

Follow-up required: Yes, the technical team needs to follow up with the customer to diagnose and fix the alarm system problem.

Ambiguities: None identified in the conversation.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Customer's home security alarm system is going off randomly at night without apparent cause.",
    "resolution": "Agent suggests running a diagnostic to check for sensor malfunction and transfers customer to technical team.",
    "followUpRequired": true,
    "followUpDetails": "Technical team to diagnose and resolve the issue with the customer's alarm system."
  },
  "status": "COM

どれも良さそうです。次に、少し曖昧さを含む通話記録で試し、JSON の `ambiguities` に反映されるか確認します：


In [ ]:
ambiguous_call = """
エージェント: アクリーム・スマートホームサポートへお電話ありがとうございます。アレックスです。本日はどのようにお手伝いできますか？
顧客: こんにちは、アレックスさん。スマートロックに問題があります。正常に動作していません。
エージェント: それは申し訳ございません。スマートロックで何が起きているか、もう少し詳しく教えていただけますか？
顧客: ええと、家を出る時にロックされないことがあります。スマートフォンに関係しているかもしれませんが、よくわかりません。
エージェント: 承知いたしました。ロックされないというのは、自動ロック機能に反応しないということですか、それともアプリから手動でロックしようとしているのですか？
顧客: うーん、両方だと思います。時々は一方が動いて、時々はもう一方が動きます。一貫していません。
エージェント: わかりました。スマートフォンに関係している可能性があるとおっしゃいましたが、ドアに近づいた方がうまく動くなどのパターンはありますか？
顧客: もしかしたら？あまり注意を払っていませんでした。
エージェント: 承知いたしました。トラブルシューティングを試してみましょう。まず、お使いのスマートロックのモデルを教えていただけますか？
顧客: わかりません。6ヶ月ほど前に購入したのですが、それが参考になりますか？
エージェント: 大丈夫です。ロック本体にモデル番号が表示されていますか？
顧客: 確認する必要があります。最新モデルと仮定してもらえませんか？
エージェント: 正確なモデルがわかれば、より効果的にトラブルシューティングできます。でも、わかっている情報で続けましょう。最近ロックをリセットしてみましたか？
顧客: したと思います。いや、それはスマートサーモだったかもしれません。それにも問題があるんです。
エージェント: 承知いたしました。スマートロックの完全な診断が必要かもしれません。今すぐそのプロセスを案内してもよろしいですか？
顧客: 実は、予定があるので行かなければなりません。後で電話してもいいですか？
エージェント: もちろんです。お帰りになる前に、技術チームがより詳細なトラブルシューティングのために連絡できる連絡先番号を教えていただけますか？
顧客: はい、555...あ、待ってください。それは古い番号です。新しい番号を確認します...そうですね、時間がある時にまた電話します。
エージェント: 承知いたしました。トラブルシューティングの準備ができましたら、24時間365日対応しております。他に何かお手伝いできることはありますか？
顧客: いえ、それで大丈夫です。ありがとうございます。
エージェント: どういたしまして。アクリーム・スマートホームをご利用いただき、ありがとうございます。良い一日をお過ごしください！
"""

summarize_call_with_improved_prompt(ambiguous_call)

<thinking>
Main Issue: The customer is experiencing issues with their Acme SmartLock not consistently locking automatically or manually through the app.

Resolution: The agent attempted to troubleshoot by asking for the specific SmartLock model and suggesting a reset, but the customer had to leave before completing the troubleshooting process.

Follow-Up Required: Yes, the customer needs to call back to complete a full diagnostic and troubleshooting session with the technical team.

Ambiguities:
- The customer was unsure if the issue was related to their phone or not, suggesting a potential connectivity problem.
- The customer mentioned having issues with another Acme product (SmartTherm), but it's unclear if those issues are related to the SmartLock problem.
- The customer's contact number was not clearly provided, which could make follow-up more difficult.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "SmartLock not consistently locking automatically or manually through t

良いですね。意図どおりに動いていそうです。


---

## Edge cases

ここまでの通話記録は比較的「普通の」サポート通話でした。実運用では、要約したくない通話も出てきます。例えば：

- 接続不良の通話
- 言語の壁がある通話
- 音声が崩れて判読できない通話
- 非合理的／過度に感情的な顧客の通話

目的はサポート品質の評価なので、こうした通話を同列に要約すると分析が歪みます。

現在のプロンプトでこれらのエッジケースを流すとどうなるか、試してみましょう。下で新しい通話記録を用意します。


In [ ]:
wrong_number_call = """
エージェント: アクリーム・スマートホームサポートです。リサと申します。どのようにお手伝いできますか？
顧客: こちらはテクニカルサポートですか？
エージェント: はい、アクリーム・スマートホームデバイスのテクニカルサポートです。どのようにお手伝いできますか？
顧客: すみません、間違えました。
エージェント: 問題ありません。良い一日をお過ごしください。
"""

incomplete_call = """
エージェント: アクリーム・スマートホームサポートです。サラと申します。本日はどのようにお手伝いできますか？
顧客: そのものが動きません。
エージェント: それは申し訳ございません。どのデバイスに問題があるか、具体的に教えていただけますか？
顧客: いつものやつです。行かなければ。さようなら。
エージェント: 待ってください、もっと詳しい情報が... [通話切断]
"""

garbled_call = """
エージェント: アクリーム・スマートホームサポートへお電話ありがとうございます。アレックスです。本日はどのようにお手伝いできますか？
顧客: [聞き取れない音声]
エージェント: もしもし？いらっしゃいますか？
"""

language_barrier_call = """
エージェント: アクリーム・スマートホームサポートです。サラと申します。本日はどのようにお手伝いできますか？
顧客: [スペイン語で話している]
エージェント: 申し訳ございませんが、スペイン語は話せません。英語は話せますか？
顧客: [スペイン語を続ける]
エージェント: 少々お待ちください。通訳をおつなぎします...
"""


これらのエッジケース通話をプロンプトに通して、結果を見てみましょう：


In [66]:
summarize_call_with_improved_prompt(wrong_number_call)

<thinking>
Issue: The customer appears to have dialed the wrong number for technical support.
Resolution: Since it was a wrong number, there was no issue to resolve. The agent politely concluded the call.
Follow-up: No follow-up is required since it was a misdialed call.
Ambiguities: There are no apparent ambiguities in this brief conversation.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "The customer dialed the wrong number for technical support",
    "resolution": "The agent concluded the call politely since it was a misdialed number",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>


In [67]:
summarize_call_with_improved_prompt(incomplete_call)

<thinking>
In this transcript, the main issue is unclear as the customer does not provide sufficient details about the device they are having trouble with. The agent attempts to clarify the issue, but the customer disconnects the call before providing more information.

There is no resolution since the issue is not clearly identified. However, follow-up is required to gather more details from the customer about the specific device and the problem they are experiencing.

The key ambiguity is the lack of clarity about the device and the nature of the problem. The customer's vague statements ("the thing isn't working" and "the usual one") do not provide enough information for the agent to diagnose or resolve the issue.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Customer reported an unspecified device was not working but did not provide further details.",
    "resolution": "No resolution was possible due to lack of information from the customer.",
    "followUpRequired": tr

In [68]:
summarize_call_with_improved_prompt(garbled_call)

<thinking>
From the transcript, it appears the customer called Acme Smart Home Support, but their voice was garbled, and the agent could not understand them. With this limited information, it is unclear what the main issue or reason for the call was. There was no resolution provided, and it is ambiguous whether follow-up is required since the issue itself is unknown. The key ambiguity is the lack of clear communication from the customer, preventing the agent from understanding the problem.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Unclear due to garbled voice from the customer",
    "resolution": "No resolution provided since the issue could not be understood",
    "followUpRequired": true,
    "followUpDetails": "Agent should try to reconnect with the customer for clearer communication"
  },
  "status": "COMPLETE",
  "ambiguities": ["The customer's voice was garbled, preventing understanding of the issue"]
}
</json>


In [69]:
summarize_call_with_improved_prompt(language_barrier_call)

<thinking>
Main issue: The customer called and spoke in Spanish, but the agent could not understand Spanish.
Resolution: The agent tried to get a translator on the line to resolve the language barrier.
Follow-up required: Yes, the agent needs to connect with a Spanish translator to assist the customer.
Ambiguities: It is unclear why the customer called, as the reason for their call is not stated in the transcript.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Customer spoke in a language the agent did not understand (Spanish).",
    "resolution": "Agent attempted to get a translator to resolve the language barrier.",
    "followUpRequired": true,
    "followUpDetails": "Agent needs to connect the customer with a Spanish translator."
  },
  "status": "COMPLETE",
  "ambiguities": ["Reason for the customer's call is not stated in the transcript."]
}
</json>


残念ながら、エッジケースでも「普通の要約」を作ってしまっています。例えば次のような部分です：

>  "customerIssue": "Customer spoke in a language the agent did not understand (Spanish)."

> "customerIssue": "Unclear due to garbled voice from the customer"

> "customerIssue": "The customer dialed the wrong number for technical support"

これらの通話を要約に混ぜると、後の分析に支障が出ます。エッジケースをどう扱うかの戦略が必要です。


---

## Further prompt improvements

現状はエッジケースでも要約してしまうので、挙動を変えたいです。方針としては例えば：

- 要約不可能としてフラグ付けし、後で人間が確認できるようにする
- "technical difficulty" や "language barrier" のように別カテゴリへ分類する

ここでは簡単のため、要約不可能な通話は次の JSON を返すようにします：

```json
{
  "status": "INSUFFICIENT_DATA"
}
```

これを実現するために必要な変更：
- "INSUFFICIENT_DATA" 出力の条件を説明する指示を追加
- 要約できる／できない例と、それに対応する JSON 出力例を追加


### Updating our instructions

"INSUFFICIENT_DATA" を出すべき条件を説明するために、指示セクションに新しい文面を追加します。


In [ ]:
# 新しい内容のみ。後でプロンプト全体を見ます
new_instructions_addition = """
情報不足の基準:
   以下のいずれかの条件が満たされる場合:
   a) 通話記録のやり取りが5回未満である、または
   b) 顧客の問題が不明確である
   c) 通話が聞き取れない、不完全である、または言語の壁によって妨げられている
   次のJSONのみを返してください:
   {
     "status": "INSUFFICIENT_DATA"
   }
"""

### Adding examples

このコースで見てきた通り、例（examples）を入れるのはほぼ常に有効です。このユースケースでは特に、要約可能／不可能の両方で、望ましい出力を Claude に理解させるのに役立ちます。

プロンプトに入れられる例のセットは次です：


In [ ]:
examples_for_prompt = """
<examples>
1. 完全な対話:
<transcript>
エージェント: アクリーム・スマートホームサポートへお電話ありがとうございます。アレックスです。本日はどのようにお手伝いできますか？
顧客: こんにちは、アレックスさん。アクリーム・スマートサーモが設定した温度を維持できていません。72度に設定しているのに、家の中がもっと暑いんです。
エージェント: それは申し訳ございません。トラブルシューティングを始めましょう。スマートサーモはWi-Fiに接続されていますか？
顧客: はい、ディスプレイにWi-Fiのマークが表示されています。
エージェント: 承知いたしました。スマートサーモを再キャリブレーションしましょう。メニューボタンを5秒間長押ししてください。
顧客: わかりました。新しいメニューが表示されました。
エージェント: 完璧です。「キャリブレーション」に移動して選択を押してください。部屋の温度計に合わせて温度を調整してください。
顧客: 了解しました。79度に合わせて設定しました。
エージェント: よろしいです。選択を押して確定してください。再キャリブレーションには数分かかる場合があります。1時間後に確認して、修正されているかご確認ください。
顧客: わかりました。そうします。ご協力ありがとうございました、アレックスさん。
エージェント: どういたしまして！他に何かお手伝いできることはありますか？
顧客: いえ、それで大丈夫です。ありがとうございました。
エージェント: アクリーム・スマートホームをご利用いただき、ありがとうございます。良い一日をお過ごしください！
</transcript>

<thinking>
主な問題: スマートサーモが設定温度を維持していない
解決方法: 顧客に再キャリブレーションプロセスを案内
フォローアップ: 不要だが、顧客は1時間後に効果を確認すべき
曖昧な点: 特定されず
</thinking>

<json>
{
  "summary": {
    "customerIssue": "スマートサーモが設定温度を維持せず、設定した72度より高い温度を示している",
    "resolution": "顧客にスマートサーモの再キャリブレーションプロセスを案内",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>

2. フォローアップが必要な対話:
<transcript>
エージェント: アクリーム・スマートホームサポートです。ジェイミーと申します。どのようにお手伝いできますか？
顧客: こんにちは、新しいアクリーム・スマートカムをインストールしたのですが、Wi-Fiに接続できません。
エージェント: お手伝いさせていただきます。アクリーム・スマートホームアプリを使用していますか？
顧客: はい、スマートフォンにアプリがあります。
エージェント: よろしいです。スマートフォンが5GHzではなく2.4GHzのWi-Fiネットワークに接続されていることを確認してください。
顧客: ああ、5GHzネットワークに接続しています。切り替えるべきですか？
エージェント: はい、2.4GHzネットワークに切り替えてください。スマートカムは2.4GHzでのみ動作します。
顧客: わかりました。切り替えました。次はどうすればいいですか？
エージェント: アプリを開き、「デバイスを追加」を選択し、「スマートカム」を選択して、画面の指示に従ってください。
顧客: パスワードを求められています。
エージェント: Wi-Fiパスワードを入力すれば接続されるはずです。
顧客: まだ動作しません。エラーメッセージが表示され続けています。
エージェント: 承知いたしました。その場合、技術チームにエスカレートさせていただきます。24時間以内にご連絡いたします。
顧客: わかりました。お手伝いありがとうございました。
エージェント: どういたしまして。他に何かお手伝いできることはありますか？
顧客: いえ、今のところそれで大丈夫です。ありがとうございました。
エージェント: アクリーム・スマートホームをご利用いただき、ありがとうございます。良い一日をお過ごしください！
</transcript>

<thinking>
主な問題: 顧客が新しいスマートカムをWi-Fiに接続できない
解決方法: 初期のトラブルシューティングは成功せず、問題が技術チームにエスカレートされた
フォローアップ: 必要、技術チームが24時間以内に顧客に連絡
曖昧な点: 顧客が受け取っている特定のエラーメッセージが言及されていない
</thinking>

<json>
{
  "summary": {
    "customerIssue": "新しいスマートカムをWi-Fiに接続できない",
    "resolution": "初期のトラブルシューティングは成功せず、問題が技術チームにエスカレートされた",
    "followUpRequired": true,
    "followUpDetails": "技術チームが24時間以内に顧客に連絡し、さらなる支援を提供"
  },
  "status": "COMPLETE",
  "ambiguities": ["顧客が受け取っている特定のエラーメッセージが言及されていない"]
}
</json>

3. 情報不足:
<transcript>
エージェント: アクリーム・スマートホームサポートです。サムと申します。どのようにお手伝いできますか？
顧客: こんにちは、スマートロックが動作していません。
エージェント: それは申し訳ございません。問題についてもう少し詳しく教えていただけますか？
顧客: ただ動作しないだけです。他に何も言うことがありません。
エージェント: わかりました。いつ頃から問題に気づきましたか？そして、お使いのアクリーム・スマートロックのモデルは何ですか？
顧客: 覚えていません。聞いてください、行かなければなりません。後で電話します。
エージェント: 承知いたしました。さらなる支援が必要でしたら、24時間365日対応しております。良い一日をお過ごしください。
</transcript>

<thinking>
この通話記録は5回未満のやり取りで、顧客の問題は不明確です。顧客はスマートロックの問題について具体的な詳細を提供せず、エージェントの質問にも答えていません。この対話は完全な要約に十分な情報を提供していません。
</thinking>

<json>
{
  "status": "INSUFFICIENT_DATA"
}
</json>
</examples>
"""

例は次の3パターンをカバーしています：
* フォローアップ不要の完了通話
* フォローアップが必要で曖昧点もある完了通話
* 情報不足で要約不能な通話

例を与えるときは、入出力の多様性を確保するのが重要です。


---

## Our final prompt

最初のプロンプトに、前セクションで追加した要素を統合します：
* 情報不足通話の扱い指示
* 例のセット

これが最終プロンプトです：


In [ ]:
system = """
あなたは、通話記録から重要な情報を抽出し、構造化された形式で要約することに長けた専門のカスタマーサービス分析者です。
あなたのタスクは、カスタマーサービス通話記録を分析し、専門的なトーンを保ちながら、簡潔で正確な要約を生成することです。
"""

prompt = """
以下のカスタマーサービス通話記録を分析し、対話のJSON要約を生成してください：

<transcript>
[通話記録をここに挿入]
</transcript>

指示:
<instructions>
1. 通話記録を注意深く読んでください。
2. 通話記録を分析し、主な問題、解決方法、必要なフォローアップに焦点を当ててください。
3. 指定された構造に従って、対話の主要な側面を要約するJSONオブジェクトを生成してください。

重要なガイドライン:
- 機密性: 名前、電話番号、メールアドレスなどの特定の顧客データはすべて省略してください。
- 文字数制限: 各テキストフィールドは最大100文字に制限してください。
- 要約では専門的なトーンを保ってください。

出力形式:
以下の構造を持つJSONオブジェクトを生成してください：
<json>
{
  "summary": {
    "customerIssue": "主な問題または通話の理由の簡潔な説明",
    "resolution": "問題がどのように対処または解決されたか（該当する場合）",
    "followUpRequired": true/false,
    "followUpDetails": "必要なフォローアップアクションの説明、または不要な場合はnull"
  },
  "status": "COMPLETE",
  "ambiguities": ["会話の不明確または曖昧な点のリスト、または該当しない場合は空の配列"]
}
</json>

情報不足の基準:
   以下のいずれかの条件が満たされる場合:
   a) 通話記録のやり取りが5回未満である
   b) 顧客の問題が不明確である
   c) 通話が聞き取れない、不完全である、または言語の壁によって妨げられている
   次のJSONのみを返してください:
   {
     "status": "INSUFFICIENT_DATA"
   }

例: 
<examples>
1. 完全な対話:
<transcript>
エージェント: アクリーム・スマートホームサポートへお電話ありがとうございます。アレックスです。本日はどのようにお手伝いできますか？
顧客: こんにちは、アレックスさん。アクリーム・スマートサーモが設定した温度を維持できていません。72度に設定しているのに、家の中がもっと暑いんです。
エージェント: それは申し訳ございません。トラブルシューティングを始めましょう。スマートサーモはWi-Fiに接続されていますか？
顧客: はい、ディスプレイにWi-Fiのマークが表示されています。
エージェント: 承知いたしました。スマートサーモを再キャリブレーションしましょう。メニューボタンを5秒間長押ししてください。
顧客: わかりました。新しいメニューが表示されました。
エージェント: 完璧です。「キャリブレーション」に移動して選択を押してください。部屋の温度計に合わせて温度を調整してください。
顧客: 了解しました。79度に合わせて設定しました。
エージェント: よろしいです。選択を押して確定してください。再キャリブレーションには数分かかる場合があります。1時間後に確認して、修正されているかご確認ください。
顧客: わかりました。そうします。ご協力ありがとうございました、アレックスさん。
エージェント: どういたしまして！他に何かお手伝いできることはありますか？
顧客: いえ、それで大丈夫です。ありがとうございました。
エージェント: アクリーム・スマートホームをご利用いただき、ありがとうございます。良い一日をお過ごしください！
</transcript>

<thinking>
主な問題: スマートサーモが設定温度を維持していない
解決方法: 顧客に再キャリブレーションプロセスを案内
フォローアップ: 不要だが、顧客は1時間後に効果を確認すべき
曖昧な点: 特定されず
</thinking>

<json>
{
  "summary": {
    "customerIssue": "スマートサーモが設定温度を維持せず、設定した72度より高い温度を示している",
    "resolution": "顧客にスマートサーモの再キャリブレーションプロセスを案内",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>

2. フォローアップが必要な対話:
<transcript>
エージェント: アクリーム・スマートホームサポートです。ジェイミーと申します。どのようにお手伝いできますか？
顧客: こんにちは、新しいアクリーム・スマートカムをインストールしたのですが、Wi-Fiに接続できません。
エージェント: お手伝いさせていただきます。アクリーム・スマートホームアプリを使用していますか？
顧客: はい、スマートフォンにアプリがあります。
エージェント: よろしいです。スマートフォンが5GHzではなく2.4GHzのWi-Fiネットワークに接続されていることを確認してください。
顧客: ああ、5GHzネットワークに接続しています。切り替えるべきですか？
エージェント: はい、2.4GHzネットワークに切り替えてください。スマートカムは2.4GHzでのみ動作します。
顧客: わかりました。切り替えました。次はどうすればいいですか？
エージェント: アプリを開き、「デバイスを追加」を選択し、「スマートカム」を選択して、画面の指示に従ってください。
顧客: パスワードを求められています。
エージェント: Wi-Fiパスワードを入力すれば接続されるはずです。
顧客: まだ動作しません。エラーメッセージが表示され続けています。
エージェント: 承知いたしました。その場合、技術チームにエスカレートさせていただきます。24時間以内にご連絡いたします。
顧客: わかりました。お手伝いありがとうございました。
エージェント: どういたしまして。他に何かお手伝いできることはありますか？
顧客: いえ、今のところそれで大丈夫です。ありがとうございました。
エージェント: アクリーム・スマートホームをご利用いただき、ありがとうございます。良い一日をお過ごしください！
</transcript>

<thinking>
主な問題: 顧客が新しいスマートカムをWi-Fiに接続できない
解決方法: 初期のトラブルシューティングは成功せず、問題が技術チームにエスカレートされた
フォローアップ: 必要、技術チームが24時間以内に顧客に連絡
曖昧な点: 顧客が受け取っている特定のエラーメッセージが言及されていない
</thinking>

<json>
{
  "summary": {
    "customerIssue": "新しいスマートカムをWi-Fiに接続できない",
    "resolution": "初期のトラブルシューティングは成功せず、問題が技術チームにエスカレートされた",
    "followUpRequired": true,
    "followUpDetails": "技術チームが24時間以内に顧客に連絡し、さらなる支援を提供"
  },
  "status": "COMPLETE",
  "ambiguities": ["顧客が受け取っている特定のエラーメッセージが言及されていない"]
}
</json>

3. 情報不足:
<transcript>
エージェント: アクリーム・スマートホームサポートです。サムと申します。どのようにお手伝いできますか？
顧客: こんにちは、スマートロックが動作していません。
エージェント: それは申し訳ございません。問題についてもう少し詳しく教えていただけますか？
顧客: ただ動作しないだけです。他に何も言うことがありません。
エージェント: わかりました。いつ頃から問題に気づきましたか？そして、お使いのアクリーム・スマートロックのモデルは何ですか？
顧客: 覚えていません。聞いてください、行かなければなりません。後で電話します。
エージェント: 承知いたしました。さらなる支援が必要でしたら、24時間365日対応しております。良い一日をお過ごしください。
</transcript>

<thinking>
この通話記録は5回未満のやり取りで、顧客の問題は不明確です。顧客はスマートロックの問題について具体的な詳細を提供せず、エージェントの質問にも答えていません。この対話は完全な要約に十分な情報を提供していません。
</thinking>

<json>
{
  "status": "INSUFFICIENT_DATA"
}
</json>
</examples>
</instructions>

JSONを生成する前に、<thinking>タグ内で通話記録を分析してください。
主な問題、解決方法、フォローアップ要件、および曖昧な点の特定を含めてください。
その後、<json>タグ内にJSON出力を提供してください。
"""

上のプロンプトは長いですが、構造は次のとおりです：
- system prompt：文脈・役割・トーン
- main prompt：
    - 通話記録
    - 指示（一般指示、ガイドライン、出力要件、エッジケース条件、例）
    - 出力で使う XML タグの指定

流れを可視化すると次のようになります：

```txt
Analyze the following customer service call transcript and generate a JSON summary of the interaction:

<transcript>
[INSERT CALL TRANSCRIPT HERE]
</transcript>

<instructions>
- General instructions and guidelines
- Output JSON format description
- Insufficient data (edge-case) criteria
<examples>
varied example inputs and outputs
</examples>
</instructions>

Before generating the JSON, please analyze the transcript in <thinking> tags. 
Include your identification of the main issue, resolution, follow-up requirements, and any ambiguities. 
Then, provide your JSON output in <json> tags.

```


最終プロンプトを新しい関数でテストします（この関数は `<json>` タグ内の JSON だけを抽出します）：


In [ ]:
import re

def summarize_call_with_final_prompt(transcript):
    final_prompt = prompt.replace("[通話記録をここに挿入]", transcript)
    # API呼び出しを実行
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        system=system,
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    
    # <json>タグ間のコンテンツを抽出
    json_content = re.search(r'<json>(.*?)</json>', response.content[0].text, re.DOTALL)
    
    if json_content:
        print(json_content.group(1).strip())
    else:
        print("レスポンスにJSONコンテンツが見つかりませんでした。")

既存の通話変数のいくつかで試します：


In [77]:
summarize_call_with_final_prompt(call1)

{
  "summary": {
    "customerIssue": "Unable to turn on smart light bulb",
    "resolution": "Agent guided customer to reset the bulb by cycling power off and on",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}


In [78]:
summarize_call_with_final_prompt(call3)

{
  "summary": {
    "customerIssue": "Acme SecureHome alarm system going off randomly multiple times at night without apparent cause",
    "resolution": "Initial troubleshooting steps taken, but issue unresolved. Customer transferred to technical team for diagnostics",
    "followUpRequired": true,
    "followUpDetails": "Technical team to diagnose and resolve issue with alarm system"
  },
  "status": "COMPLETE",
  "ambiguities": []
}


次に、`ambiguities` が空でない要約になるはずの通話で試します：


In [79]:
summarize_call_with_final_prompt(ambiguous_call)

{
  "summary": {
    "customerIssue": "SmartLock not reliably locking automatically or through app, behavior is inconsistent",
    "resolution": "Troubleshooting attempted but incomplete due to lack of model details, customer had to leave",
    "followUpRequired": true,
    "followUpDetails": "Customer to call back for further troubleshooting of SmartLock issue when available"
  },
  "status": "COMPLETE",
  "ambiguities": [
    "Unclear if related SmartTherm issue mentioned",
    "SmartLock model not identified",
    "Customer's contact number not confirmed"
  ]
}


さらに、要約してほしくないエッジケース通話で試します：


In [80]:
summarize_call_with_final_prompt(garbled_call)

{
  "status": "INSUFFICIENT_DATA"
}


In [82]:
summarize_call_with_final_prompt(language_barrier_call)

{
  "status": "INSUFFICIENT_DATA"
}


In [83]:
summarize_call_with_final_prompt(incomplete_call)

{
  "status": "INSUFFICIENT_DATA"
}


良いですね。狙ったとおりの出力が得られています。さらに攻めてみましょう：


In [84]:
summarize_call_with_final_prompt("blah blah blah")

{
  "status": "INSUFFICIENT_DATA"
}


In [85]:
summarize_call_with_final_prompt("")

{
  "status": "INSUFFICIENT_DATA"
}


素晴らしい。エッジケースも含めてきちんと処理できています。

---


## Wrap up


このレッスンでは、カスタマーサポート通話記録を要約する複雑なプロンプトを作るプロセスを一通り見ました。使ったテクニックを振り返ります：

* **System Prompt**：全体の文脈と役割を設定
* **Structured Input**：通話記録を XML タグで冒頭に配置
* **Clear Instructions**：注目点と出力の作り方を詳細に指示
* **Output Formatting**：JSON 構造を指定し、一貫してパースしやすい出力に
* **Handling Edge Cases**：情報不足通話を識別する条件を追加
* **Examples**：複数シナリオの例で望む出力を誘導
* **Thinking Aloud**：`<thinking>` で分析させてから JSON を出させる

これらにより、多様な通話に対して構造化要約を作りつつ、エッジケースも適切に扱えるプロンプトになりました。このアプローチは要約以外の複雑なユースケースにも応用できます。

**Important Note:** ただし、このプロンプトはまだ本番投入できる状態ではありません。現状は少数例の目視確認に基づくプロトタイプです。実運用の通話は多様で予測不能なので、定量メトリクスを伴う包括的な評価（evals）が不可欠です。データ駆動の堅牢な評価が、良さそうな試作品と信頼できる本番品質の間のギャップを埋めます。
